In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/03/24 12:33:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/24 12:33:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
!wget https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/355125/anti-social-behaviour-order-statistics-court-level-breaches-0113.csv

In [4]:
!wc -l ASBO-2001-2013.csv

5548 ASBO-2001-2013.csv


In [5]:
df = spark.read \
    .option("header", "true") \
    .csv('ASBO-2001-2013.csv')

In [6]:
df.schema

StructType([StructField('Year of Breach', StringType(), True), StructField('Age group breach 2', StringType(), True), StructField('Court_breach', StringType(), True), StructField('Sex', StringType(), True), StructField('ASBOs_breached', StringType(), True)])

In [7]:
!head -n 1001 ASBO-2001-2013.csv > head.csv

In [8]:
import pandas as pd

In [9]:
df_pandas = pd.read_csv('head.csv')

In [12]:
df_pandas

,Year of Breach,Age group breach 2,Court_breach,Sex,ASBOs_breached
0,2002,18+,"Abingdon, Didcot and Wantage Magistrates' Court",1,1
1,2004,18+,"Abingdon, Didcot and Wantage Magistrates' Court",1,1
2,2005,18+,"Abingdon, Didcot and Wantage Magistrates' Court",2,1
3,2008,18+,"Abingdon, Didcot and Wantage Magistrates' Court",2,1
4,2009,18+,"Abingdon, Didcot and Wantage Magistrates' Court",1,2
...,...,...,...,...,...
995,2007,'10-17,Derby & South Derbyshire Magistrates' Court,1,1
996,2007,18+,Derby & South Derbyshire Magistrates' Court,1,6
997,2003,18+,Derby Crown Court,1,1
998,2004,18+,Derby Crown Court,1,1


In [10]:
df_pandas.dtypes

Year of Breach         int64
Age group breach 2    object
Court_breach          object
Sex                    int64
ASBOs_breached         int64
dtype: object

Integer - 4 bytes
Long - 8 bytes

In [14]:
from pyspark.sql import types

In [15]:
schema = types.StructType([
    types.StructField('Year of Breach', types.IntegerType(), True),
    types.StructField('Age group breach 2', types.StringType(), True),
    types.StructField('Court_breach', types.StringType(), True),
    types.StructField('Sex', types.IntegerType(), True),
    types.StructField('ASBOs_breached', types.IntegerType(), True),
])

In [16]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('ASBO-2001-2013.csv')

In [17]:
df = df.repartition(24)

In [18]:
df.write.parquet('ASBO/2001-2013/')

In [19]:
df = spark.read.parquet('ASBO/2001-2013/')

In [20]:
df.printSchema()

root
 |-- Year of Breach: integer (nullable = true)
 |-- Age group breach 2: string (nullable = true)
 |-- Court_breach: string (nullable = true)
 |-- Sex: integer (nullable = true)
 |-- ASBOs_breached: integer (nullable = true)



SELECT * FROM df WHERE Sex =  1

In [21]:
from pyspark.sql import functions as F

In [22]:
df.show()

+--------------+------------------+--------------------+---+--------------+
|Year of Breach|Age group breach 2|        Court_breach|Sex|ASBOs_breached|
+--------------+------------------+--------------------+---+--------------+
|          2009|               18+|Sussex (Eastern) ...|  1|             8|
|          2012|               18+|Batley and Dewsbu...|  1|             1|
|          2010|               18+|Northampton Magis...|  1|             4|
|          2008|               18+|Northampton Magis...|  1|             3|
|          2009|            '10-17|North Durham Yout...|  1|             2|
|          2002|            '10-17|City of Salford Y...|  1|             1|
|          2003|            '10-17|South East Wiltsh...|  1|             1|
|          2011|               18+|Barking and Dagen...|  1|             2|
|          2010|               18+|Reading Magistrat...|  1|             3|
|          2013|               18+|Norfolk Magistrat...|  1|             1|
|          2